In [36]:
%pylab inline
import numpy as np
import sklearn.linear_model as linear
from sklearn.model_selection import KFold
from scipy.stats import ttest_rel
import pandas as pd
from sklearn import preprocessing
import torch as th
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


In [13]:
df = pd.read_csv('wdbc.data', header=None)

In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
diagnosis_encoder = preprocessing.LabelEncoder()
diagnosis_encoder.fit(['B', 'M'])
df[1] = diagnosis_encoder.transform(df[1])
df = df.drop([0], axis=1)

In [23]:
df.head(10)

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,1,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,1,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,1,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,1,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [28]:
y = df[1].values
X = df.drop([1], axis=1).values

In [38]:
class Net(nn.Module):
    def __init__(self, n_inputs):
        super().__init__()
        self.l1 = nn.Linear(n_inputs, n_inputs // 2)
        self.act1 = nn.PReLU()
        self.l2 = nn.Linear(n_inputs // 2, 1)
        self.act2 = nn.Sigmoid()
    
    def forward(self, inputs, asnumpy=False):
        res = self.l1(inputs)
        res = self.act1(res)
        res = self.l2(res)
        res = self.act2(res)
        if asnumpy:
            res = res.detach().numpy()
        return res
    
    def predict(self, inputs):
        inputs = th.tensor(inputs, dtype=th.float32)
        probs = self.forward(inputs)
        return np.array(probs > 0.5, dtype=np.int)
        
    def compute_loss(self, X, y):
        probs = self.forward(X)
        loss_components = y * th.log(probs) + (1 - y) * th.log(1 - probs)
        return th.mean(loss_components)

In [52]:
def fit_net(net, X, y, epochs=200, lr=0.01):
    optimizer = optim.RMSprop(net.parameters(), lr=lr)
    X = th.tensor(X, dtype=th.float32)
    y = th.tensor(y, dtype=th.float32)
    for _ in range(epochs):
        optimizer.zero_grad()
        loss = net.compute_loss(X, y)
        loss.backward()

In [53]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)

5

In [54]:
n_cols = X.shape[1]

In [68]:
lr_res = []
net_res = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    lr = linear.LogisticRegression()
    net = Net(n_cols)
    
    lr.fit(X_train, y_train)
    fit_net(net, X_train, y_train)
    
    lr_proba = lr.predict_proba(X_test)
    lr_res.append(metrics.roc_auc_score(y_test, lr_proba[:, 1]))
    net_res.append(metrics.roc_auc_score(y_test, net.forward(th.tensor(X_test, dtype=th.float32), True)))
    
lr_res = np.array(lr_res)
net_res = np.array(net_res)

/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarn

In [69]:
print('Mean AUC of Logistic Regressio ', np.mean(lr_res), ' Std: ', np.std(lr_res))
print('Mean AUC of Logistic Regressio ', np.mean(net_res), ' Std: ', np.std(net_res))

Mean AUC of Logistic Regressio  0.9945146822954138  Std:  0.0020718453470754137
Mean AUC of Logistic Regressio  0.3518318615242947  Std:  0.3536915876775104


In [70]:
print(ttest_rel(lr_res, net_res))

Ttest_relResult(statistic=3.6400255153403482, pvalue=0.0219641592277414)


Differences between Logisitic Regression and Net are statistically different. Morover, Logistic regression performed better; hence we should use Logistic Regression

The better performance of Logistic Regression indicates the the problem is linearly solvable